In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pynamit
from polplot import Polarplot
import xarray as xr
import warnings
import ipywidgets as widgets
from IPython.display import display
from pynamit.math.constants import RE
from pynamit.math.constants import mu0

warnings.filterwarnings("ignore", message="No contour levels were found within the data range.")


B_kwargs = {
    "cmap": plt.cm.bwr,
    "levels": np.linspace(-100, 100, 22) * 1e-9,
    "extend": "both",
}
eqJ_kwargs = {"colors": "black", "levels": np.r_[-210:220:20] * 1e3}
jr_kwargs = {
    "cmap": plt.cm.bwr,
    "levels": np.linspace(-0.95, 0.95, 22) / 6 * 1e-6,
    "extend": "both",
}
Phi_kwargs = {"colors": "black", "levels": np.r_[-212.5:220:5] * 1e3}
W_kwargs = {"colors": "green", "levels": Phi_kwargs["levels"] / 4}


%matplotlib widget


settings = xr.load_dataset("results_mage_2011_settings.ncdf")

sh_basis = pynamit.SHBasis(settings.Nmax, settings.Mmax)

# conversion factors:
m_ind_to_Br = -(settings.RI**2) * sh_basis.laplacian(settings.RI)
m_imp_to_jr = settings.RI / mu0 * sh_basis.laplacian(settings.RI)
W_to_dBr_dt = 1 / settings.RI
m_ind_to_Jeq = -settings.RI / mu0 * (2 * sh_basis.n + 1)


mainfield = pynamit.Mainfield(
    kind=settings.mainfield_kind, epoch=settings.mainfield_epoch, hI=(settings.RI - RE) * 1e-3
)


lat, lon = np.linspace(-89.9, 89.9, 60), np.linspace(-180, 180, 100)
lat, lon = np.meshgrid(lat, lon)
plt_grid = pynamit.Grid(lat=lat, lon=lon)
plt_evaluator = pynamit.BasisEvaluator(sh_basis, plt_grid)


## MAP PROJECTION:
global_projection = ccrs.PlateCarree(central_longitude=0)

fig = plt.figure(figsize=(10, 6))

paxn_imposed = Polarplot(plt.subplot2grid((2, 4), (0, 0)))
paxs_imposed = Polarplot(plt.subplot2grid((2, 4), (0, 1)))
paxn_induced = Polarplot(plt.subplot2grid((2, 4), (0, 2)))
paxs_induced = Polarplot(plt.subplot2grid((2, 4), (0, 3)))
gax_imposed = plt.subplot2grid((2, 2), (1, 0), projection=global_projection, rowspan=2)
gax_induced = plt.subplot2grid((2, 2), (1, 1), projection=global_projection, rowspan=2)

for ax in [gax_imposed, gax_induced]:
    ax.coastlines(zorder=2, color="grey")

plt.subplots_adjust(top=0.89, bottom=0.095, left=0.025, right=0.95, hspace=0.0, wspace=0.185)


state_data = xr.load_dataset("results_mage_2011_state.ncdf")

indexer = state_data.indexes["time"].get_indexer([0], method="nearest")
subset = state_data.isel(time=indexer[0])
m_imp = subset.SH_m_imp.values
m_ind = subset.SH_m_ind.values
Phi_coef = subset.SH_Phi.values * settings.RI
W_coef = subset.SH_W.values * settings.RI


jr = plt_evaluator.G.dot(m_imp * m_imp_to_jr)
Jeq = plt_evaluator.G.dot(m_ind * m_ind_to_Jeq)
Br = plt_evaluator.G.dot(m_ind * m_ind_to_Br)
Phi = plt_evaluator.G.dot(Phi_coef)
W = plt_evaluator.G.dot(W_coef)


contours = {}
## GLOBAL PLOTS
contours["jr "] = gax_imposed.contourf(
    lon, lat, jr.reshape(lon.shape), transform=ccrs.PlateCarree(), **jr_kwargs
)
contours["Jeq"] = gax_induced.contour(
    lon, lat, Jeq.reshape(lon.shape), transform=ccrs.PlateCarree(), **eqJ_kwargs
)
contours["Br"] = gax_induced.contourf(
    lon, lat, Br.reshape(lon.shape), transform=ccrs.PlateCarree(), **B_kwargs
)
contours["Phi"] = gax_imposed.contour(
    lon, lat, Phi.reshape(lon.shape), transform=ccrs.PlateCarree(), **Phi_kwargs
)
contours["W"] = gax_imposed.contour(
    lon, lat, W.reshape(lon.shape), transform=ccrs.PlateCarree(), **W_kwargs
)


# north:
nnn = lat > 50
contours["jr_n"] = paxn_imposed.contourf(
    lat[nnn], lon[nnn] / 15, jr.reshape(lon.shape)[nnn], **jr_kwargs
)
contours["Phi_n"] = paxn_imposed.contour(
    lat[nnn], lon[nnn] / 15, Phi.reshape(lon.shape)[nnn], **Phi_kwargs
)
contours["W_n"] = paxn_imposed.contour(
    lat[nnn], lon[nnn] / 15, W.reshape(lon.shape)[nnn], **W_kwargs
)
contours["Jeq_n"] = paxn_induced.contour(
    lat[nnn], lon[nnn] / 15, Jeq.reshape(lon.shape)[nnn], **eqJ_kwargs
)
contours["Br_n"] = paxn_induced.contourf(
    lat[nnn], lon[nnn] / 15, Br.reshape(lon.shape)[nnn], **B_kwargs
)


# south:
sss = lat < -50
contours["jr_s"] = paxs_imposed.contourf(
    lat[sss], lon[sss] / 15, jr.reshape(lon.shape)[sss], **jr_kwargs
)
contours["Phi_s"] = paxs_imposed.contour(
    lat[sss], lon[sss] / 15, Phi.reshape(lon.shape)[sss], **Phi_kwargs
)
contours["W_s"] = paxs_imposed.contour(
    lat[sss], lon[sss] / 15, W.reshape(lon.shape)[sss], **W_kwargs
)
contours["Jeq_s"] = paxs_induced.contour(
    lat[sss], lon[sss] / 15, Jeq.reshape(lon.shape)[sss], **eqJ_kwargs
)
contours["Br_s"] = paxs_induced.contourf(
    lat[sss], lon[sss] / 15, Br.reshape(lon.shape)[sss], **B_kwargs
)


# Define the update function
def update(time):
    """Update the plot with the given time.

    Parameters
    ----------
    time : int
        The time index.
    """
    global contours
    for contour in contours.values():
        for c in contour.collections:
            c.remove()

    indexer = state_data.indexes["time"].get_indexer([time], method="nearest")
    subset = state_data.isel(time=indexer[0])
    m_imp = subset.SH_m_imp.values
    m_ind = subset.SH_m_ind.values
    Phi_coef = subset.SH_Phi.values * settings.RI
    W_coef = subset.SH_W.values * settings.RI
    jr = plt_evaluator.G.dot(m_imp * m_imp_to_jr)
    Jeq = plt_evaluator.G.dot(m_ind * m_ind_to_Jeq)
    Br = plt_evaluator.G.dot(m_ind * m_ind_to_Br)
    Phi = plt_evaluator.G.dot(Phi_coef)
    W = plt_evaluator.G.dot(W_coef)

    ## GLOBAL PLOTS
    contours["jr "] = gax_imposed.contourf(
        lon, lat, jr.reshape(lon.shape), transform=ccrs.PlateCarree(), **jr_kwargs
    )
    contours["Jeq"] = gax_induced.contour(
        lon, lat, Jeq.reshape(lon.shape), transform=ccrs.PlateCarree(), **eqJ_kwargs
    )
    contours["Br"] = gax_induced.contourf(
        lon, lat, Br.reshape(lon.shape), transform=ccrs.PlateCarree(), **B_kwargs
    )
    contours["Phi"] = gax_imposed.contour(
        lon, lat, Phi.reshape(lon.shape), transform=ccrs.PlateCarree(), **Phi_kwargs
    )
    contours["W"] = gax_imposed.contour(
        lon, lat, W.reshape(lon.shape), transform=ccrs.PlateCarree(), **W_kwargs
    )

    # north:
    contours["jr_n"] = paxn_imposed.contourf(
        lat[nnn], lon[nnn] / 15, jr.reshape(lon.shape)[nnn], **jr_kwargs
    )
    contours["Jeq_n"] = paxn_induced.contour(
        lat[nnn], lon[nnn] / 15, Jeq.reshape(lon.shape)[nnn], **eqJ_kwargs
    )
    contours["Br_n"] = paxn_induced.contourf(
        lat[nnn], lon[nnn] / 15, Br.reshape(lon.shape)[nnn], **B_kwargs
    )
    contours["Phi_n"] = paxn_imposed.contour(
        lat[nnn], lon[nnn] / 15, Phi.reshape(lon.shape)[nnn], **Phi_kwargs
    )
    contours["W_n"] = paxn_imposed.contour(
        lat[nnn], lon[nnn] / 15, W.reshape(lon.shape)[nnn], **W_kwargs
    )

    # south:
    contours["jr_s"] = paxs_imposed.contourf(
        lat[sss], lon[sss] / 15, jr.reshape(lon.shape)[sss], **jr_kwargs
    )
    contours["Jeq_s"] = paxs_induced.contour(
        lat[sss], lon[sss] / 15, Jeq.reshape(lon.shape)[sss], **eqJ_kwargs
    )
    contours["Br_s"] = paxs_induced.contourf(
        lat[sss], lon[sss] / 15, Br.reshape(lon.shape)[sss], **B_kwargs
    )
    contours["Phi_s"] = paxs_imposed.contour(
        lat[sss], lon[sss] / 15, Phi.reshape(lon.shape)[sss], **Phi_kwargs
    )
    contours["W_s"] = paxs_imposed.contour(
        lat[sss], lon[sss] / 15, W.reshape(lon.shape)[sss], **W_kwargs
    )

    fig.canvas.draw_idle()


# Create the slider widget
slider = widgets.IntSlider(min=0, max=state_data.time.max(), step=10, value=0)
display(slider)


# Define the function to handle slider value changes
def on_value_change(change):
    """Handle the change of the slider value.

    Parameters
    ----------
    change : dict
        The change dictionary.
    """
    update(change["new"])


# Attach the observer to the slider
slider.observe(on_value_change, names="value")

plt.show()

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import pynamit
from pynamit.math.constants import RE

state_data = xr.load_dataset("results_mage_2011_state.ncdf")
steady_state_data = xr.load_dataset("results_mage_2011_steady_state.ncdf")
settings = xr.load_dataset("results_mage_2011_settings.ncdf")
stations = pd.read_csv(
    "hdstations.csv"
)  # a list of some of the 1-sec stations from SuperMAG (there are more!)
glon, glat = stations["GEOLON"].values, stations["GEOLAT"].values

ground_grid = pynamit.Grid(lat=glat, lon=glon)
sh_basis = pynamit.SHBasis(settings.Nmax, settings.Mmax)
ground_evaluator = pynamit.BasisEvaluator(sh_basis, ground_grid)

m_ind_to_Bh_ground = -(sh_basis.n + 1) * (RE / settings.RI) ** sh_basis.n
m_ind_to_Br_ground = sh_basis.n * (sh_basis.n + 1) * (RE / settings.RI) ** (sh_basis.n - 1)

# calculate the time series:
m_ind = state_data.SH_m_ind.values.T
m_ind_steady_state = steady_state_data.SH_m_ind.values.T

Br = (ground_evaluator.G * m_ind_to_Br_ground.reshape((1, -1))).dot(m_ind)
Bh = (-ground_evaluator.G_grad * m_ind_to_Bh_ground.reshape((1, -1))).dot(m_ind)

Br_steady_state = (ground_evaluator.G * m_ind_to_Br_ground.reshape((1, -1))).dot(m_ind_steady_state)
Bh_steady_state = (-ground_evaluator.G_grad * m_ind_to_Bh_ground.reshape((1, -1))).dot(m_ind_steady_state)

fig, axes = plt.subplots(ncols=4, nrows=3)

for i in range(len(stations)):
    ax = axes.flatten()[i]
    ax.plot(state_data.time.values, Br[i] * 1e9, label="$B_r$")
    ax.plot(state_data.time.values, Bh[0, i] * 1e9, label="$B_\\theta$")
    ax.plot(state_data.time.values, Bh[1, i] * 1e9, label=r"$B_\phi$")
    ax.set_title(stations.iloc[i].IAGA)

axes[0, 0].legend(frameon=False)

plt.show()